In [11]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from gensim.models import Word2Vec

In [2]:
data = pd.read_csv('george_wassouf_lyrics_dataset.csv')
data.head()

,Track Title,Track Lyrics,Unnamed: 2
0,Aah Habaieb - آه حبايب,اه حبايب بس فين الحب غايب\r\nوليه يا قلبي ما ...,NaN
1,Aalem Albi El Shok - علم قلبي الشوق,\r\nيا ليلي يا ليلي يا ليل\r\n\r\nعلم قلبي الش...,NaN
2,Addi Eli Kanou - أدي اللي كانوا,أدي اللي كانوا يا هوا أقرب ما لينا\r\nواللي ما...,NaN
3,Alb Al Asheq Daleiloh - قلب العاشق دليله,قلب العاشق دليله\r\nقلب العاشق دليله\r\n\r\nقل...,NaN
4,Al Forqah Saabah - الفرقة صعبة,مش قادر اصبرعغيابا\r\nكل ليله بجمع دموعي\r\nو ...,NaN


In [4]:
# Number of songs
no_of_songs = data.shape[0]
print(no_of_songs)
# Number of attributes
no_of_attributes = data.shape[1]
print(no_of_attributes)
# Attribute names
attrs = data.columns.tolist()
print(attrs)

78
3
['Track Title', 'Track Lyrics', 'Unnamed: 2']


In [ ]:
track_lyrics_df = data['Track Lyrics'].to_frame()
print(track_lyrics_df.head())

                                        Track Lyrics
0   اه حبايب بس فين الحب غايب\r\nوليه يا قلبي ما ...
1  \r\nيا ليلي يا ليلي يا ليل\r\n\r\nعلم قلبي الش...
2  أدي اللي كانوا يا هوا أقرب ما لينا\r\nواللي ما...
3  قلب العاشق دليله\r\nقلب العاشق دليله\r\n\r\nقل...
4  مش قادر اصبرعغيابا\r\nكل ليله بجمع دموعي\r\nو ...


In [12]:
stop_words = set(stopwords.words('arabic'))
data['Track Lyrics'] = track_lyrics_df['Track Lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
data.head()

,Track Title,Track Lyrics,Unnamed: 2
0,Aah Habaieb - آه حبايب,اه حبايب فين الحب غايب وليه قلبي لكش نايب شافو...,NaN
1,Aalem Albi El Shok - علم قلبي الشوق,ليلي ليلي ليل قلبي الشوق الشوق عيني السهر خدني...,NaN
2,Addi Eli Kanou - أدي اللي كانوا,أدي اللي كانوا هوا أقرب لينا واللي هانوا هوا ي...,NaN
3,Alb Al Asheq Daleiloh - قلب العاشق دليله,قلب العاشق دليله قلب العاشق دليله قلب العاشق د...,NaN
4,Al Forqah Saabah - الفرقة صعبة,مش قادر اصبرعغيابا ليله بجمع دموعي بسهر دمعات ...,NaN
